# OLDStyleCLIPS:  Empowering Image Editing with Textual Guidance using StyleGAN Latent Space Optimization, CLIP, and Image Segmentation

## Introduction:

This notebook serves as the main playground for the project, enabling you to perform image editing tasks using the StyleGAN2 generetor trained on the `ffhq` dataset. You have two options for editing:

- Image based on a seed:
We generate a random latent vector, based on the provided seed to have an initial image that we can perform edits on.
You can explore different variations of the generated image using editing parameters.

- Your image:
[Invert](https://colab.research.google.com/github/khalilacheche/StyleCLIP/blob/main/notebooks/inversion_playground.ipynb) your own images to obtain their latent space representations for the generator. Once the image inverted, you can upload the `generated_latent.pt` and specify the path of the latent vector in the form below. 
This allows you to perform edits on your own images using the obtained latent vectors.

## Setup:

1- (This may take minutes) Run the first cell to initialize the environment.
This will clone the repository and download the required model weights for editing.
Image Editing UI:

2- Run the second cell to set up the user interface for image editing.
The UI allows you to select different editing parameters, explained in detail below.
Visualizing the Effect of Latent Direction Scaling:

3- Run the last cell to set up a UI for visualizing the impact of scaling the optimized latent direction on the original image.


In [1]:
#@title Setup (may take a few minutes)
!git clone https://github.com/khalilacheche/StyleCLIP.git

import os
os.chdir(f'./StyleCLIP')

!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# downloads StyleGAN's weights and facial recognition network weights
ids = ['1EM87UquaoQmk17Q8d5kYIAHqu0dkYqdT', '1N0MZSqPRJpLfP4mFQCS14ikrVSe8vQlL']
for file_id in ids:
  downloaded = drive.CreateFile({'id':file_id})
  downloaded.FetchMetadata(fetch_all=True)
  downloaded.GetContentFile(downloaded.metadata['title'])

Cloning into 'StyleCLIP'...
remote: Enumerating objects: 1016, done.
remote: Counting objects: 100% (398/398), done.
remote: Compressing objects: 100% (208/208), done.
remote: Total 1016 (delta 241), reused 284 (delta 176), pack-reused 618
Receiving objects: 100% (1016/1016), 260.33 MiB | 21.13 MiB/s, done.
Resolving deltas: 100% (386/386), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-2ikrmb4m
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-2ikrmb4m
  Resolved https://github.com/openai/CLIP.git to commit a9b1bf5920416aaeaec965c25dd9e8f98c864f16
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1

## Editing an Image

In this UI, there are 4 main sections for editing parameters:

### 1. Editing Parameters



#### a. Editing Text
- Choose the text caption for the desired image output.

#### b. Semantic Parts for Localization Score
- Select multiple semantic parts of the image to focus on for the localization score.
- Use CMD/Ctrl or Shift keys for multiple selections.
- The localization score measures the extent of editing outside the selected region of interest.

### 2. Optimization Parameters

#### a. Latent Space Selection
- Choose between `W+` latent space and `StyleSpace` for optimization.
- Note: `StyleSpace` provides more disentanglement, but reduce the L2 lambda compared to `W+` space.

#### a. CLIP Loss Lambda
- Set the scaling factor for the CLIP loss, which measures the similarity between input text and output image.

#### c. L2 Loss Lambda
- Adjust the scaling factor for the norm of the found latent direction.
- This influences the magnitude of the applied edits.

#### d. ID Loss Lambda
- Set the scaling factor for the identity loss, which measures the similarity of identities between the original and output images.

#### e. Localization Loss Lambda
- Adjust the scaling factor for the localization loss.

#### f. Optimization Steps
- Specify the number of iterations for optimizing and updating the latent direction.

### 3. Input Image Parameters

#### a. Use Seed
- Check this option to use an automatically generated image based on StyleGAN with a specific seed.

#### b. Seed
- Provide the seed value used to generate the input image.

#### c. Latent Path
- Enter the local path of the `.pt` file containing the latent code of an inverted image.
- This is required for performing edits on custom images.

### Output Parameters

#### a. Save Image Every Step
- If checked, saves every intermediary image outputted by each optimization step.
- The images are saved in the `/results` subdirectory with the format `000i.jpg`, where `i` represents the iteration index.
- If not checked, an image is saved every 20 iterations.

#### b. Save Segmentation Output
- If checked, saves the output of the segmentation model along with the intermediary generated image.
- The segmentation output is saved as `seg_000i.jpg`.


Once you have selected the parameters, you can click on the "Generate Image" button to start the editing. This operation takes some time (around 20 seconds) and the edited image is displayed alongside the original one.

You can directly change the parameters again and click the generate button, without re-running the cell.

It is worth noting that sometimes, depending on your internet connection and the load on collab's servers, it can take some time for the editing to start (signaled by the "Starting optimization..." log)

In [2]:
#@title Optimization
import ipywidgets as widgets
from IPython.display import display
import torch
from optimization.run_optimization import main
from argparse import Namespace
from torchvision.utils import make_grid
from torchvision.transforms import ToPILImage
import numpy as np

clip_lambda_widget = widgets.FloatText(
    value=1,
    description='CLIP lambda:',
    disabled=False
)
l2_lambda_widget = widgets.FloatText(
    value=0.0004,
    description='L2 lambda:',
    disabled=False
)
loc_lambda_widget = widgets.FloatText(
    value=0.00001,
    description='Loc lambda:',
    disabled=False
)
id_lambda_widget = widgets.FloatText(
    value=0.005,
    description='ID lambda:',
    disabled=False
)
use_stylespace_widget = widgets.Checkbox(
    value=True,
    description='Stylespace',
    disabled=False
)
optimization_steps_widget = widgets.IntText(
    value=40,
    description='Optimization Steps:',
    disabled=False
)

optimization_parameters_widgets = [clip_lambda_widget, l2_lambda_widget,loc_lambda_widget,id_lambda_widget,use_stylespace_widget,optimization_steps_widget] #lambdas, use stylespace, optimization steps
optimization_parameters_box = widgets.VBox(optimization_parameters_widgets)




editing_text_widget = widgets.Text(
    value='A person with black hair',
    placeholder='Type a description of the output image',
    description='Output image desc:',
    disabled=False
)

semantic_parts_options = ["mouth","skin","eyes","nose","ears","eye_brows","hat","hair","neck"]
semantic_parts_widget = widgets.SelectMultiple(
    options=semantic_parts_options,
    value=["hair"],
    #rows=10,
    description='Semantic Parts',
    disabled=False
)
editing_parameters_widgets = [editing_text_widget,semantic_parts_widget] # text, semantic part
editing_parameters_box = widgets.VBox(editing_parameters_widgets)


use_seed_widget = widgets.Checkbox(
    value=True,
    description='Use seed?',
    disabled=False
)

seed_widget = widgets.IntText(
    value=1,
    description='Seed:',
    disabled=False
)

latent_path_widget = widgets.Text(
    value=None,
    placeholder='latents/example.pth',
    description='Latent vector path',
    disabled=False
)
input_image_parameters_widgets = [use_seed_widget,seed_widget,latent_path_widget] # use seed? seed, latent path
input_image_parameters_box = widgets.VBox(input_image_parameters_widgets)



create_video_widget = widgets.Checkbox(
    value=False,
    description='Save output at every step?',
    disabled=False
)

export_segmentation_out_widget = widgets.Checkbox(
    value=False,
    description='Export segmentation output?',
    disabled=False
)


output_parameters_widgets = [create_video_widget,export_segmentation_out_widget] #create_video? export_segmentation_image? range of scaling, 
output_parameters_box = widgets.VBox(output_parameters_widgets)

accordion = widgets.Accordion(children=[editing_parameters_box,optimization_parameters_box,input_image_parameters_box, output_parameters_box])
accordion.set_title(0, 'Editing Parameters')
accordion.set_title(1, 'Optimization Parameters')
accordion.set_title(2, 'Input Image Parameters')
accordion.set_title(3, 'Output Parameters')

generate_button_widget = widgets.Button(
    description='Generate Image',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to generate',
    icon='check'
)
out = widgets.Output()


results = None

def button_action(b):
  global results
  out.clear_output()
  with out:
    print("Starting optimization...")
    
    args = {
    "description": editing_text_widget.value,
    "ckpt": "stylegan2-ffhq-config-f.pt",
    "stylegan_size": 1024,
    "lr_rampup": 0.05,
    "lr": 0.1,
    "step": optimization_steps_widget.value,
    "clip_lambda": clip_lambda_widget.value,
    "l2_lambda": l2_lambda_widget.value,
    "id_lambda": id_lambda_widget.value,
    "loc_lambda": loc_lambda_widget.value,
    'work_in_stylespace': use_stylespace_widget.value,
    "latent_path": None if use_seed_widget.value else latent_path_widget.value,
    "truncation": 0.7,
    "save_intermediate_image_every": 1 if create_video_widget.value else 20,
    "results_dir": "results",
    "ir_se50_weights": "model_ir_se50.pth",
    "semantic_parts":semantic_parts_widget.value,
    "export_segmentation_image": export_segmentation_out_widget.value
    }
    if (use_seed_widget.value):
      torch.manual_seed(seed_widget.value)
    else:
      if not os.path.isfile(latent_path_widget.value):
        print("Latent vector path is not a file!")
        return
    
    
    # run the optimization
    results = main(Namespace(**args))
    orig_res_imgs = torch.cat(
            [
                results["orig_image"],
                results["gen_image"],
            ]
    )
    result_image = ToPILImage()(make_grid(orig_res_imgs.detach().cpu(), normalize=True, scale_each=True, range=(-1, 1), padding=0))
    h, w = result_image.size
    display(result_image.resize((h // 2, w // 2)))
    
generate_button_widget.on_click(button_action)

In [3]:
display(accordion,generate_button_widget,out)

Accordion(children=(VBox(children=(Text(value='A person with black hair', description='Output image desc:', pl…

Button(description='Generate Image', icon='check', style=ButtonStyle(), tooltip='Click to generate')

Output()

## Scaling the Optimized Latent Direction

To observe the effect of the found latent direction, you can scale it using the following steps:

1. Run the `create_image_output_editing_form` cell to ensure you have the latest results from the previous optimization.

2. Adjust the slider to select the number of images you want to generate.

3. Use the range slider to set the boundaries of the scaling. The minimum and maximum selected values determine the scaling range.

4. Click the button to generate the specified number of images. The scaling will uniformly grow from the minimum selected value to the maximum selected value, progressing from left to right.

In [6]:
#@title Scaling the latent vector (rerun the cell below to update the result from optimization)
def prepare_images_for_display(imgs):
  concatenated = torch.cat(imgs)
  result_image = ToPILImage()(make_grid(concatenated.detach().cpu(), normalize=True, scale_each=True, range=(-1, 1), padding=0))
  h, w = result_image.size
  im=result_image.resize((h // 2, w // 2))
  return im
def get_images_scaled(generator,latent_init,latent_dir,min_scale,max_scale,num_scales):
  imgs = []
  for scale in np.linspace (min_scale,max_scale, num_scales):
    is_stylespace = use_stylespace_widget.value
    new_latent = (
            [(latent_init[c] + scale * latent_dir[c]) for c in range(len(latent_init))]
            if (is_stylespace)
            else latent_init + scale * latent_dir
        )
    result_gen, _ = generator(
                    [new_latent],
                    input_is_latent=True,
                    randomize_noise=False,
                    input_is_stylespace=is_stylespace,
                )
    imgs.append(result_gen["image"])
  return imgs


def create_image_output_editing_form(results):
  num_scaling_steps_widget = widgets.IntSlider(
      value=3,
      min=3,
      max=7,
      step=2,
      description='Number of images',
      disabled=False,
      continuous_update=False,
      orientation='horizontal',
      readout=True,
      readout_format='d'
  )


  scaling_range_value_widget = widgets.FloatRangeSlider(
      value=[-1, 1],
      min=-2.0,
      max=2.0,
      step=0.001,
      description='Scaling range:',
      disabled=False,
      continuous_update=True,
      orientation='horizontal',
      readout=True,
      readout_format='.2f',
  )
  scale_button_widget =  widgets.Button(
    description='Generate Image',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to generate',
    icon='check'
)
  v_box = widgets.VBox([num_scaling_steps_widget,scaling_range_value_widget,scale_button_widget])
  out_scale = widgets.Output()
  def scale_button_action(b):
    if (results is None):
      return
    out_scale.clear_output(True)
    with out_scale:
      min_scale, max_scale = scaling_range_value_widget.value
      num_scales = num_scaling_steps_widget.value
      imgs = get_images_scaled(results["generator"],results["orig_latent"],results["latent_dir"],min_scale,max_scale,num_scales)
      display(prepare_images_for_display(imgs))
      torch.cuda.empty_cache()

  scale_button_widget.on_click(scale_button_action)
  display(v_box,out_scale)
  with out_scale:
    scale_button_action(None)



  

In [7]:
create_image_output_editing_form(results)

Output()